In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('StructureAPI').config('spark.some.config.option', 'some-value').getOrCreate()

In [2]:
from pyspark.sql.functions import col, column, expr
from pyspark.sql import functions as f

In [3]:
df = spark.read.json('movies.json')

In [4]:
df.show()

+----------------+--------------------+--------------------+----+
|            cast|              genres|               title|year|
+----------------+--------------------+--------------------+----+
|              []|                  []|After Dark in Cen...|1900|
|              []|                  []|Boarding School G...|1900|
|              []|                  []|Buffalo Bill's Wi...|1900|
|              []|                  []|              Caught|1900|
|              []|                  []|Clowns Spinning Hats|1900|
|              []|[Short, Documentary]|Capture of Boer B...|1900|
|              []|                  []|The Enchanted Dra...|1900|
|   [Paul Boyton]|                  []|   Feeding Sea Lions|1900|
|              []|            [Comedy]|How to Make a Fat...|1900|
|              []|                  []|     New Life Rescue|1900|
|              []|                  []|    New Morning Bath|1900|
|              []|                  []|Searching Ruins o...|1900|
|         

In [5]:
df.printSchema()

root
 |-- cast: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [6]:
# 1. Show the number of distinct fimls in the dataset
df.distinct().count()

28789

In [7]:
# 2. Count the number of films released during the years 2012 and 2015 (included)
df.filter((col('year') >= 2012) & (col('year') <= 2015)).count()

1015

In [8]:
# 3. Show the year in which the number of movies released is highest. Only one highest year is expected in the output.
df.groupBy('year').count().sort('count', ascending=False).limit(1).show()

+----+-----+
|year|count|
+----+-----+
|1919|  634|
+----+-----+



In [9]:
# 4. Show the list of movies such that for each film, 
# the number of actors/actresses is at least five, and the number of genres it belongs to is at most two genres.
df.filter((f.size('cast') >= 5) & (f.size('genres') <= 2)).show()

+--------------------+----------------+--------------------+----+
|                cast|          genres|               title|year|
+--------------------+----------------+--------------------+----+
|[Earle Foxe, Alie...|         [Drama]|  A Desperate Chance|1913|
|[Charlotte Burton...|         [Drama]|    The Archeologist|1914|
|[Charlotte Burton...|         [Drama]|At the Potter's W...|1914|
|[Herbert Tracey, ...|        [Comedy]|    Back to the Farm|1914|
|[Charlotte Burton...|              []|    The Beggar Child|1914|
|[William Garwood,...|              []|       Billy's Rival|1914|
|[B. Reeves Eason,...|         [Drama]| Break, Break, Break|1914|
|[Charlotte Burton...|              []|       The Butterfly|1914|
|[Charlotte Burton...|       [Western]|Calamity Anne's L...|1914|
|[Charlie Chaplin,...|        [Comedy]|    The Star Boarder|1914|
|[Sydney Ayres, Ja...|              []|A Story of Little...|1914|
|[Sydney Ayres, Pe...|              []|The Story of the ...|1914|
|[Charlott

In [10]:
# 5. Show all the movies whose names are longest.
df.filter(f.length('title') == df.select(f.max(f.length('title'))).collect()[0][0]).show()


+----+------+--------------------+----+
|cast|genres|               title|year|
+----+------+--------------------+----+
|  []|    []|Cornell-Columbia-...|1901|
+----+------+--------------------+----+



In [11]:
# 6. Show the movies whose name contains the word “fighting” (case-insensitive). 
df.filter(f.lower(f.col('title')).contains('fighting')).show()

+--------------------+---------------+--------------------+----+
|                cast|         genres|               title|year|
+--------------------+---------------+--------------------+----+
|[Bessie Love, Ann...|[Comedy, Drama]|  A Fighting Colleen|1919|
|[Blanche Sweet, R...|      [Western]|     Fighting Cressy|1919|
|[Harry T. Morey, ...|        [Drama]|    Fighting Destiny|1919|
|[Tom Mix, Teddy S...|      [Western]|   Fighting for Gold|1919|
|[Jack Perrin, Hoo...|      [Western]|  The Fighting Heart|1919|
|[Art Acord, Mildr...|      [Western]|   The Fighting Line|1919|
|[William Duncan, ...|       [Action]|  The Fighting Guide|1922|
|[Tom Mix, Patsy R...|      [Western]| The Fighting Streak|1922|
|[Richard Barthelm...|   [Historical]|  The Fighting Blade|1923|
|[Ernest Torrence,...|       [Comedy]| The Fighting Coward|1924|
|[Jack Hoxie, Hele...|      [Western]|       Fighting Fury|1924|
|[Pat O'Malley, Ma...|        [Drama]|The Fighting Adve...|1924|
|[Fred Thomson, Ha...|   

In [12]:
# Show the list of distinct genres in the dataset.
df.select(f.explode('genres')).distinct().show()

+-------------+
|          col|
+-------------+
|        Crime|
|      Romance|
|     Thriller|
|      Slasher|
|Found Footage|
|    Adventure|
|         Teen|
| Martial Arts|
|       Sports|
|        Drama|
|          War|
|  Documentary|
|       Family|
|      Fantasy|
|       Silent|
|     Disaster|
|        Legal|
|      Mystery|
| Supernatural|
|     Suspense|
+-------------+
only showing top 20 rows



In [13]:
# 8. List all movies in which the actor Harrison Ford has participated.
# df2 = df.withColumn('individual cast', f.explode('cast'))

# df2.filter(f.col('individual cast').contains('Harrison Ford')).show()

In [14]:
# 8. List all movies in which the actor Harrison Ford has participated.
df.filter(f.array_contains('cast', 'Harrison Ford')).show()

+--------------------+-----------------+--------------------+----+
|                cast|           genres|               title|year|
+--------------------+-----------------+--------------------+----+
|[Constance Talmad...|[Romance, Comedy]|Experimental Marr...|1919|
|[Constance Talmad...|         [Comedy]| Happiness a la Mode|1919|
|[Constance Talmad...|         [Comedy]|Romance and Arabella|1919|
|[Vivian Martin, H...|         [Comedy]|      The Third Kiss|1919|
|[Harrison Ford, C...|         [Comedy]|The Veiled Adventure|1919|
|[Constance Talmad...|         [Comedy]|          Who Cares?|1919|
|[Vivian Martin, H...|          [Drama]|You Never Saw Suc...|1919|
|[Norma Talmadge, ...|          [Drama]| The Wonderful Thing|1921|
|[Alma Rubens, Har...|        [Mystery]|      Find the Woman|1922|
|[Constance Talmad...|          [Drama]| The Primitive Lover|1922|
|[Norma Talmadge, ...| [Romance, Drama]|     Smilin' Through|1922|
|[Helen Jerome Edd...|          [Drama]|     When Love Comes|1

In [21]:
# 9. List all movies in which the actors/actresses whose names include the word “Lewis“ (case-insensitive) have participated.
df2 = df.withColumn('individual cast', f.explode('cast'))

df2.filter(f.col('individual cast').contains('Lewis')).show()

+--------------------+-----------+--------------------+----+---------------+
|                cast|     genres|               title|year|individual cast|
+--------------------+-----------+--------------------+----+---------------+
|[Charlotte Burton...|         []|       The Butterfly|1914|      Ida Lewis|
|[Pearl White, She...|    [Drama]|The Exploits of E...|1914|  Sheldon Lewis|
|[Charlotte Burton...|   [Comedy]| Mein Lieber Katrina|1914|      Ida Lewis|
|[Norma Talmadge, ...|    [Drama]|      Going Straight|1916|    Ralph Lewis|
|[Dorothy Gish, Ra...|    [Drama]|Gretchen the Gree...|1916|    Ralph Lewis|
|[Ben Lewis, Bessi...|  [Western]|     A Sister of Six|1916|      Ben Lewis|
|[Gail Kane, Lewis...|    [Drama]| The Bride's Silence|1917|  Lewis J. Cody|
|    [Mitchell Lewis]|    [Drama]|Nine-Tenths of th...|1918| Mitchell Lewis|
|[Mitchell Lewis, ...|    [Drama]|The Faith of the ...|1919| Mitchell Lewis|
|[Mary Pickford, R...|   [Comedy]|         The Hoodlum|1919|    Ralph Lewis|

In [22]:
# 10. Show top five actors/actresses that have participated in most movies.
df2.groupBy('individual cast').count().sort('count', ascending=False).limit(5).show()

+----------------+-----+
| individual cast|count|
+----------------+-----+
|    Harold Lloyd|  190|
|     Hoot Gibson|  142|
|      John Wayne|  136|
|Charles Starrett|  116|
|    Bebe Daniels|  103|
+----------------+-----+

